In [15]:
pip install TextBlob

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import nltk
from nltk .corpus import stopwords
from nltk import word_tokenize
from textblob import TextBlob
import string
import re
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Antes de esto lo que he hecho ha sido:
    #Unir tweets_positivos y tweets_negativos en tweets

In [4]:
def eliminar_punto_coma(archivo_csv, archivo_salida):
    with open(archivo_csv, 'r') as archivo_entrada, open(archivo_salida, 'w', newline='') as archivo_salida:
        lector_csv = csv.reader(archivo_entrada)
        escritor_csv = csv.writer(archivo_salida)

        for fila in lector_csv:
            fila_sin_punto_coma = [re.sub(r';', '', columna) for columna in fila]
            escritor_csv.writerow(fila_sin_punto_coma)


In [6]:
eliminar_punto_coma('tweets.csv', 'tweets2.csv')

In [7]:
df = pd.read_csv('tweets2.csv', header=None, names=['text'], on_bad_lines='skip', encoding= 'unicode_escape')
df.head(10)

,text
0,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...
2,my whole body feels itchy and like its on fire
3,@Kwesidei not the whole crew
4,Need a hug
5,@Tatiana_K nope they didn't have it
6,@twittera que me muera ?
7,spring break in plain city... it's snowing
8,I just re-pierced my ears
9,@caregiving I couldn't bear to watch it. And ...


In [1]:
# Creamos la funcion 'limpiar_texto' que va a remover las stopwords y otros elementos  
def limpiar_texto(text):  
    lemmatizer = nltk.stem.WordNetLemmatizer()
    stopWords = set(stopwords.words('english'))
    text = str(text)
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text) #Elimina las menciones
    text = re.sub(r'RT[|\s]', ' ', text) # Elimina los RTs
    text = re.sub(r'#', ' ', text) #Elimina los hashtags
    text = re.sub(r'https?:\/\/\S+', ' ', text) #Elimina los enlaces

    pattern = r'''(?x)                 # Activa la opcion que permite expresiones regulares
               (?:[A-Z]\.)+            # abreviaturas
               | \w+(?:-\w+)*          # Palabras con guiones externos que pueden ser opcionales
               | \$?\d+(?:\.\d+)?%?    # Monedas y porcentajes
               | \.\.\.                # Elipsis
              | [][.,;"'?():-_`]       #Tokens separados
               '''
    words = nltk.regexp_tokenize(text, pattern)
    delete_punt = re.compile('[%s]' % re.escape(string.punctuation)) # Elimina signos de puntuacion
    stripped = [delete_punt.sub('', w) for w in words]
    delete_stopwords = [w for w in stripped if  w.lower() not in stopWords] # Elimina las stopwords
    lemmatized_words = [lemmatizer.lemmatize(w) for w in delete_stopwords]

    return (" ".join(lemmatized_words))

In [10]:
df['tweets_transform'] = df['text'].apply(limpiar_texto)
df

,text,tweets_transform
0,is upset that he can't update his Facebook by ...,upset update Facebook texting might cry resu...
1,@Kenichan I dived many times for the ball. Man...,dived many times ball Managed save 50 rest go...
2,my whole body feels itchy and like its on fire,whole body feels itchy like fire
3,@Kwesidei not the whole crew,whole crew
4,Need a hug,Need hug
...,...,...
769,@lisaholmes you can buy my fiances duplex in T...,buy fiances duplex Terwillegar
770,@apleaforaaron i could really eat some cheese ...,could really eat cheese hankering austrian smo...
771,@ddlovato I am so FUCKING jealous of you girl....,FUCKING jealous girl fun though xxx
772,@ThaStevieG my bunnys name was Lucy you can g...,bunnys name Lucy go ahead guess named lmao


In [15]:
#Con esta función, mediante la libreria TextBlob analizamos la polaridad. Este valor, que oscila entre 1 y -1 nos indica que
#el tweet es positivo si su valor es cercano al 1, neutro si el valor es 0 y negativo si su valor es cercano al -1.
def analisis_sentimiento(text):
  text = str(text)
  if text != '':
    analisis = TextBlob(text) 
    analisis = analisis.sentiment
    sentiment = analisis.polarity
    return sentiment

In [16]:
df['polarity'] = df['tweets_transform'].apply(analisis_sentimiento)
df

,text,tweets_transform,polarity
0,is upset that he can't update his Facebook by ...,upset update Facebook texting might cry resu...,0.00
1,@Kenichan I dived many times for the ball. Man...,dived many times ball Managed save 50 rest go...,0.50
2,my whole body feels itchy and like its on fire,whole body feels itchy like fire,0.20
3,@Kwesidei not the whole crew,whole crew,0.20
4,Need a hug,Need hug,0.00
...,...,...,...
769,@lisaholmes you can buy my fiances duplex in T...,buy fiances duplex Terwillegar,0.00
770,@apleaforaaron i could really eat some cheese ...,could really eat cheese hankering austrian smo...,0.20
771,@ddlovato I am so FUCKING jealous of you girl....,FUCKING jealous girl fun though xxx,-0.15
772,@ThaStevieG my bunnys name was Lucy you can g...,bunnys name Lucy go ahead guess named lmao,0.60


In [18]:
#Guardamos en un nuevo csv
df.to_csv('tweets_polarizados.csv', sep=',', index=False)
df

,text,tweets_transform,polarity
0,is upset that he can't update his Facebook by ...,upset update Facebook texting might cry resu...,0.00
1,@Kenichan I dived many times for the ball. Man...,dived many times ball Managed save 50 rest go...,0.50
2,my whole body feels itchy and like its on fire,whole body feels itchy like fire,0.20
3,@Kwesidei not the whole crew,whole crew,0.20
4,Need a hug,Need hug,0.00
...,...,...,...
769,@lisaholmes you can buy my fiances duplex in T...,buy fiances duplex Terwillegar,0.00
770,@apleaforaaron i could really eat some cheese ...,could really eat cheese hankering austrian smo...,0.20
771,@ddlovato I am so FUCKING jealous of you girl....,FUCKING jealous girl fun though xxx,-0.15
772,@ThaStevieG my bunnys name was Lucy you can g...,bunnys name Lucy go ahead guess named lmao,0.60


In [72]:
#CSV con etiquetas correspondientes
rangos = [(-1, -0.6), (-0.6, -0.2), (-0.2, 0.2), (0.2, 0.6), (0.6, 1)]
etiquetas = ['Hater', 'Molesto', 'Neutro', 'Contento', 'Muy Feliz']

data = []
with open('tweets_polarizados.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)
        
with open('tweets_etiquetados.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Iterar sobre los datos y asignar las etiquetas correspondientes
    for row in data:
        try:
            value = float(row[-1])  # Convertir a valor numérico
            for i, (start, end) in enumerate(rangos):
                if start <= value < end:
                    row[-1] = etiquetas[i]
                    break
        except ValueError:
            # Manejo de excepción para valores no numéricos
            row[-1] = 'etiqueta'

        writer.writerow(row)

In [73]:
#Cuantas filas no están etiquetadas
with open('tweets_etiquetados.csv', 'r') as archivo_csv:
    lector_csv = csv.reader(archivo_csv)

    # Inicializa el contador de filas no válidas
    contador_filas_no_validas = 0

    # Itera sobre cada fila del archivo CSV
    for fila in lector_csv:
        # Obtén el último valor de la fila
        ultimo_valor = fila[-1]

        # Verifica si el último valor no está en la lista de valores válidos
        valores_validos = ['Hater', 'Molesto', 'Neutro', 'Contento', 'Muy Feliz']
        if ultimo_valor not in valores_validos:
            contador_filas_no_validas += 1

# Imprime el resultado
print(f"Número de filas no válidas: {contador_filas_no_validas}")

Número de filas no válidas: 18


In [74]:
#Eliminar filas incorrectas
with open('tweets_etiquetados.csv', 'r') as archivo_entrada, open('tweets_etiquetados_bien.csv', 'w', newline='') as archivo_salida:
    lector_csv = csv.reader(archivo_entrada)
    escritor_csv = csv.writer(archivo_salida)

    # Lee la primera fila y la escribe en el archivo de salida sin cambios
    primera_fila = next(lector_csv)
    escritor_csv.writerow(primera_fila)

    # Itera sobre cada fila del archivo de entrada
    for fila in lector_csv:
        # Obtén el último valor de la fila
        ultimo_valor = fila[-1]

        # Verifica si el último valor está en la lista de valores válidos
        valores_validos = ['Hater', 'Molesto', 'Neutro', 'Contento', 'Muy Feliz']
        if ultimo_valor in valores_validos:
            # Escribe la fila en el archivo de salida
            escritor_csv.writerow(fila)